In [ ]:
import cv2
import mediapipe as mp
import pandas as pd
import numpy as np
import os

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils


def extract_hand_landmarks(image_path, label=None, output_csv='hand_keypoints.csv', draw=False):

    hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.5)

    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Could not read image from path: {image_path}")
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    results = hands.process(image_rgb)

    if not results.multi_hand_landmarks:
        print(f"No hands detected in {image_path}")
        return None

    hand_landmarks = results.multi_hand_landmarks[0]
    landmarks = []
    for lm in hand_landmarks.landmark:
        landmarks.append(lm.x)
        landmarks.append(lm.y)

    data_row = np.array(landmarks)

    if draw:
        annotated = image.copy()
        mp_drawing.draw_landmarks(
            annotated, hand_landmarks, mp_hands.HAND_CONNECTIONS)
        cv2.imshow("Hand Landmarks", annotated)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    df = pd.DataFrame([data_row])
    if label is not None:
        df['label'] = label  # !!!!!!!!! append gesture label at the end

    if not os.path.exists(output_csv):
        df.to_csv(output_csv, index=False)
    else:
        df.to_csv(output_csv, mode='a', header=False, index=False)

    print(f"Saved landmarks for {image_path} to {output_csv}")
    return data_row